In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import requests
from bs4 import BeautifulSoup
import json
#url編碼
import urllib

keyword = input("請輸入關鍵字:")
limit = 50 #回傳數量
max_price = 100000 #最高價格
rate = 4 #4星以上
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36',
    'x-api-source': 'pc',
    'referer': f'https://shopee.tw/search?keyword={urllib.parse.quote(keyword)}'
}

#cookies
s = requests.Session()
url = 'https://shopee.tw/api/v4/search/product_labels'
r = s.get(url, headers=headers)
#print(r)

#搜尋頁網址
url = f"https://shopee.tw/api/v4/search/search_items/?by=relevancy&keyword={urllib.parse.quote(keyword)}&limit={limit}&newest=0&order=desc&page_type=search&price_max={max_price}&rating_filter={rate}&version=2"
re = s.get(url, headers=headers)
#print(re)

#確認資料無誤
if re.status_code == requests.codes.ok:
    data = re.json()

#總資料數
totalcount = data['query_rewrite']['ori_total_count']
#分頁（蝦皮每頁包含五十筆資料 ref:https://ithelp.ithome.com.tw/articles/10232930)
total_pg = (totalcount // 50) + 1
data['totalPage'] = total_pg
print(f"total count: {totalcount}")
print(f"total page: {total_pg}")

#對每個分頁爬蟲
for num in range(0, total_pg):
    #print("\nPage", num)
    pg_url = f"https://shopee.tw/api/v4/search/search_items/?by=relevancy&keyword={urllib.parse.quote(keyword)}&limit=50&newest={num*50}&order=desc&page_type=search&version=2"
    #page = num + 1
    
    resp = requests.get(pg_url, headers=headers)
    doc = resp.json()
    position = 0
    count = 0
    
    #擷取分頁內每個商品的資訊
    for d in doc['items']:
        #print(d)
        productid = d['item_basic']['itemid']
        name = d['item_basic']['name']
        shopid = d['item_basic']['shopid']
        item_url = f"https://shopee.tw/product/{shopid}/{productid}" #商品網址

        #商品名稱和網址
        print(name)
        print(item_url)
        
        #商品內頁
        re_item_url = f"https://shopee.tw/api/v4/item/get?itemid={productid}&shopid={shopid}" #get的網址
        re_item = requests.get(re_item_url, headers=headers)
        imf = re_item.json()
        #print(imf)
        
        # 運費優惠內頁#
        re_shipping_url = f"https://shopee.tw/api/v4/pdp/get_shipping?buyer_zipcode=&city=%E5%A4%A7%E5%AE%89%E5%8D%80&district=&itemid={productid}&shopid={shopid}&state=%E8%87%BA%E5%8C%97%E5%B8%82&town=" #get的網址
        re_shipping = requests.get(re_shipping_url, headers=headers)
        ims = re_shipping.json()
        
        #每個細項名稱+運費#
        for i, item in enumerate(ims['data']['ungrouped_channel_infos']):
            #if item['ungrouped_channel_infos'] != None :
            print(f"  第{i+1}張  {str(item['name'])}運送方式   運費{int(item['min_price']/100000)}元")
            
        
        #每個細項名稱+賣場優惠卷+價格
        for i, item in enumerate(imf['data']['shop_vouchers']):
            if item['discount_value'] != None and item['discount_value'] > 0:
                print(f"  第{i+1}張折價券  滿{int(item['min_spend']/100000)}  折底{int(item['discount_value']/100000)}元")
            elif item['discount_percentage'] != None and item['discount_percentage'] > 0:
                print(f"  第{i+1}張折價券  滿{int(item['min_spend']/100000)}  打{int(100 - item['discount_percentage'])}折  最高折抵{int(item['discount_cap']/100000)}元")
        
        #組合優惠
        if imf['data']['bundle_deal_info'] != None:
            print(f"  組合優惠：{imf['data']['bundle_deal_info']['bundle_deal_label']}")
    
        for i, item in enumerate(imf['data']['models']):
            #過濾已經沒貨的
            if item['stock'] == 0:
                continue
            if len(imf['data']['models']) == 1:
                print(f"\tprice: {int(item['price']/100000)}\tstock: {item['stock']}")
            else:
                print(f"\t{item['name']} price: {int(item['price']/100000)}\tstock: {item['stock']}")
                
       
        
        

請輸入關鍵字:掃地機器人
total count: 3000
total page: 61
【ECOVACS科沃斯】DEEBOT X1 OMNI全能旗艦掃拖機器人自動集塵&回洗拖布/熱風烘乾/自動補水/銀離子除菌
https://shopee.tw/product/143159162/16449587312
  第1張  賣家宅配運送方式   運費0元
	DEEBOT X1 OMNI限時特賣 price: 29888	stock: 416
	DEEBOT X1 OMNI price: 29888	stock: 20
【ECOVACS 科沃斯】DEEBOT N9+自動回洗風乾掃拖一體智能機器人(自動回洗抹布才乾淨/雙盤旋轉下壓拖地技術)
https://shopee.tw/product/143159162/14346434826
  第1張  賣家宅配運送方式   運費0元
	DEEBOT N9+ price: 15900	stock: 66
	DEEBOT N9+限時特賣 price: 15900	stock: 622
Roborock G10 石頭掃拖機器人自動洗拖布版 (小米生態鏈)【新品預購】
https://shopee.tw/product/104788746/14978691285
  第1張  賣家宅配運送方式   運費0元
	【活動價】 price: 21999	stock: 2674
【ECOVACS科沃斯】DEEBOT T10+智慧掃拖自動集塵版(60天自動集塵/首創航天級導航/全屋散香/內建語音助手
https://shopee.tw/product/143159162/17349602317
  第1張  賣家宅配運送方式   運費0元
	DEEBOT T10 PLUS限時特賣 price: 18888	stock: 573
	DEEBOT T10 PLUS price: 18888	stock: 99
智能掃地小超人【小寶購物】 三合一智慧型 吸塵器 智能 掃地機 家用清潔器 掃地 吸地 母親節 掃地機器人0201003
https://shopee.tw/product/2319776/9305900841
  第1張  7-11運送方式   運費60元
  第2張  萊爾富運送方式   運費0元
  第3張  全家運送方式   運費6

  第1張  賣家宅配運送方式   運費0元
	price: 17999	stock: 10
💖免運 現貨💖一鍵啟動 掃地機器人 吸塵器 掃地機 毛髮拖地 擦地機 頭髮 掃地 寵物毛髮吸收器 寵物吸塵器
https://shopee.tw/product/571112062/14532349772
  第1張  7-11運送方式   運費0元
  第2張  全家運送方式   運費0元
  第3張  黑貓宅急便運送方式   運費90元
  第1張折價券  滿599  打92折  最高折抵0元
  第2張折價券  滿899  打92折  最高折抵0元
  第3張折價券  滿1299  打92折  最高折抵100元
  組合優惠：2 件 9.8折
	黑色 price: 229	stock: 4
【牛牛柑仔店】米家掃地機器人 耗材 小米掃地機器人 配件 石頭掃地機器人 濾網 1個
https://shopee.tw/product/2518629/1350963772
  第1張  7-11運送方式   運費60元
  第2張  萊爾富運送方式   運費50元
  第3張  全家運送方式   運費60元
  第4張  OK Mart運送方式   運費45元
  第5張  蝦皮店到店運送方式   運費19元
  第6張  中華郵政運送方式   運費80元
	邊刷兩支 price: 50	stock: 940
	濾網一個 price: 40	stock: 685
	主刷一支 price: 80	stock: 941
結帳5%蝦幣回饋~【免運/24H出貨】 小米 米家掃拖機器人pro 小米 掃地機器人
https://shopee.tw/product/308649071/5384619337
  第1張  賣家宅配運送方式   運費0元
  第1張折價券  滿100  折底30元
	米家3D掃拖機器人pro 福利機 price: 11000	stock: 1
	米家3D掃拖機器人pro(陸版) price: 11690	stock: 3
台灣現貨 小米 米家 G1 主刷 滾刷 邊刷 掃地機器人 掃拖機器人 掃地機 副廠 配件 耗材
https://shopee.tw/product/8083452/15006190480
  第1張  7-11運送方式   運費60元
  第2

  第1張  7-11運送方式   運費60元
  第2張  黑貓宅急便運送方式   運費170元
  第3張  賣家宅配運送方式   運費180元
  第1張折價券  滿999  折底20元
  第2張折價券  滿499  折底10元
  第3張折價券  滿1999  折底50元
	⚫黑色⚫ price: 599	stock: 35
	⚪白色⚪ price: 599	stock: 42
【歌林自動機器人掃地機(USB充電)】掃地機器人 掃地機 充電式掃地機 吸塵器 自動掃地機 智慧型掃地機
https://shopee.tw/product/280567591/5644292066
  第1張  7-11運送方式   運費60元
  第2張  萊爾富運送方式   運費0元
  第3張  全家運送方式   運費60元
  第4張  OK Mart運送方式   運費0元
  第5張  蝦皮店到店運送方式   運費0元
  第6張  中華郵政運送方式   運費90元
  第7張  賣家宅配運送方式   運費80元
  第1張折價券  滿1299  折底40元
  第2張折價券  滿999  折底30元
  第3張折價券  滿499  折底20元
  第4張折價券  滿399  折底10元
	price: 789	stock: 923
［Electrolux］全新掃地機器人 PUREi9 2代（藍）Pi92-6STN原廠公司貨 免運優惠中
https://shopee.tw/product/10867828/11904675356
  第1張  賣家宅配運送方式   運費0元
	PI92-6STN price: 12900	stock: 1
	Pi92-6STN 加購耗材組 price: 15400	stock: 2
(2入組) 小米 石頭 小瓦 米家 掃地機器人 配件 耗材 邊刷 三腳邊刷 五腳邊刷 清潔刷
https://shopee.tw/product/8083452/12101073085
  第1張  7-11運送方式   運費60元
  第2張  萊爾富運送方式   運費50元
  第3張  全家運送方式   運費60元
  第4張  OK Mart運送方式   運費45元
  第5張  蝦皮店到店運送方式   運費0元
  第6張  賣家宅配運送方式   